In [295]:
#Read Input & Assign Variables
import time
import csv
import math
with open('speed_to_data_map.csv', newline='') as mapping:
    speed_map = tuple(csv.reader(mapping))
    
speed_data_map = []
for i in range (0,len(speed_map)):
    speed_data_map = (*speed_data_map, int(speed_map[i][1]))


In [296]:
def rank(data,K,init_speed,factor):
    index_map = []
    time = []
    
    # for i in range (0,K):
    #     if instance == 0:
    #         time.append(data[i]/init_speed[i])
    #     else:
    #         time.append(data[i]/(init_speed[i]*(1-(factor[i]/100))))
    
    for i in range (0,K):
        time.append(data[i]/(init_speed[i]*(1-(factor[i]/100))))
    
    for i in range (0,K):
        if max(time)!=0:
            max_index = (time.index(max(time)))+1
            index_map.append(max_index)
            time[max_index-1] = -1
    return index_map

In [297]:
def target_factor(user_index_map,instance,flag,factor,init_speed,total_data,data_remaining):
    sent_data = 0
    collocated_factor_list = [0]*instance
    print("instance: ", instance)
    print("index_map", user_index_map)
    for i in range (0,instance):
        collocated_factor_list[i] = factor[user_index_map[i]-1]/100
   
    for i in range (0,instance):
        collocated_factor = collocated_factor_list[i] * sum((collocated_factor_list[:i]+collocated_factor_list[i+1:]))
        speed = init_speed[user_index_map[i]-1] * (1-collocated_factor)
        if collocated_factor > 1:
            speed = 0
        speed_floor = math.floor(speed + 10e-6)
        if (data_remaining[user_index_map[i]-1] <= speed_data_map[speed_floor]):
            sent_data = sent_data + data_remaining[user_index_map[i]-1]
        else:
            sent_data = sent_data + speed_data_map[speed_floor]

        # if flag == 1:
        #     data_remaining[index_map[i]-1] = data_remaining[index_map[i]-1] - speed_data_map[speed_floor]
        #     if (data_remaining[index_map[i]-1] < 0):
        #         data_remaining[index_map[i]-1] = 0
        #     avg_speed[index_map[i]-1] = ((avg_speed[index_map[i]-1])*instance[index_map[i]-1] + speed)/(instance[index_map[i]-1]+1)
        #     instance[index_map[i]-1] = instance[index_map[i]-1] + 1

    target_curr = (sent_data/total_data)*100
    return target_curr


In [298]:
def write(tc,M,N,grid,avg_speed,best_speed,data_loss,total_data,alpha,start_time):
    with open(str(tc)+'.csv', 'w', newline='') as file:
        writer = csv.writer(file)
    
        for i in range (0,M):
            write_data = []
            for j in range (0,N):
                if (grid[j][i] == 0):
                    write_data.append("-")
                else:
                    write_data.append("U"+str(grid[j][i]))
            writer.writerow(write_data)
            
        penalty = sum(list(data_loss))/total_data
        data_loss.append(penalty)
        writer.writerow(data_loss)
        
        obj_func = sum(list(avg_speed))/best_speed
        avg_speed.append(obj_func)
        writer.writerow(avg_speed)
        
        score = []
        score.append(obj_func - (alpha*penalty))
        writer.writerow(score)
        
        execution_time = []
        temp = math.floor((time.time() - start_time)*1000)
        execution_time.append(temp)
        writer.writerow(execution_time)
        
                
            

In [299]:
def queue(M,rank_index_map, user_index_map,flag,data_remaining,target_ideal,margin_high,margin_low,grid,factor,init_speed,total_data):
    print("user map: ", user_index_map)
    print("data rem in queue: ", data_remaining)
    # for k in range (0,instance):
    #     if sum(list(data_remaining))>0:
    #         index_map_temp[k] = rank_index_map[-k]
    
    for i in range(1,M+1): 
        instance = i
        user_index_map[i-1] = rank_index_map[1-i]
        target_covered = target_factor(user_index_map,instance,flag,factor,init_speed,total_data,data_remaining)
        print("target covered: ", target_covered)
        print("low: ", margin_low*target_ideal)
        print("high: ", margin_high*target_ideal)
        # if (margin_low*target_ideal <= target_covered <= margin_high*target_ideal):
        #     print("1")
        # else:
        #     instance = instance + 1
        #     target_covered = target_factor(user_index_map,instance,flag,factor,init_speed,total_data,data_remaining)
        #     print("target covered: ", target_covered)
    
    

In [300]:
def main():
    for tc in range (3,4):
        start_time = time.time()
        with open('tc'+str(tc)+'.csv', newline='') as input:
        #with open('toy_testcase.csv', newline='') as input:
            data = tuple(csv.reader(input))

        M = int(data[0][0])
        N = int(data[0][1])
        K = int(data[1][0])
        alpha = float(data[2][0])
        
        init_speed = ()
        data_size = ()
        factor = ()
        
        for i in range (3,3+K):
            init_speed = (*init_speed, int(data[i][1]))
            data_size = (*data_size, int(data[i][2]))
            factor = (*factor, int(data[i][3]))
     
        total_data = sum(list(data_size))
        best_speed = sum(list(init_speed))
        data_remaining = list(data_size)

        grid = [[0]*M]*N
        avg_speed = [0]*K
        instance = [0]*K
        data_loss = [0]*K
        
        
        target_ideal = 100/N
        margin_low = 1
        margin_high = 2.15
        print("Target: ", target_ideal, "low: ", margin_low*target_ideal, "high: ", margin_high*target_ideal)
        print("M: ", M, "N: ", N)
        print("K: ", K)
        user_index_map = [0]*M
        #instance = 1  
        for i in range (0,N):       #Columns 
            #index_map_temp = [0]*M
            flag = 0 
            rank_index_map = rank(data_remaining.copy(),K,init_speed,factor)         #shows the index of largest and the subsequent largest data_size  
            queue(M,rank_index_map, user_index_map,flag, data_remaining,target_ideal,margin_high,margin_low,grid,factor,init_speed,total_data)
                # 
                # print("target_factor_now: ", target_factor_temp)
                # if (margin_low*target_ideal <= target_factor_temp <= margin_high*target_ideal): #or (((sum(list(data_remaining)))/total_data)*100 <= target_ideal):
                #     flag = 1
                #     count = count + 1
                #     user_index_map = index_map_temp.copy()
                #     print("-----------flagged index ",count, "-------------", user_index_map)
                #     target_factor(M,init_speed,factor,total_data,user_index_map,j+1,flag,avg_speed,instance,data_remaining)
                #     grid[i] = user_index_map.copy()
                #     break
            print("N: ",  "data_rem at end of N:", i, data_remaining)
            print("grid: ", grid)
        data_loss = data_remaining.copy()
        write(tc,M,N,grid,avg_speed,best_speed,data_loss,total_data,alpha,start_time)

In [301]:
if __name__ == '__main__':
    main()
    


Target:  4.0 low:  4.0 high:  8.6
M:  16 N:  25
K:  40
user map:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
data rem in queue:  [9823, 4591, 6869, 9293, 6908, 5723, 6679, 9625, 8774, 7698, 2432, 8756, 8959, 6304, 9495, 7530, 17854, 9063, 4471, 8343, 4585, 9088, 3171, 5976, 4585, 7232, 2787, 8475, 9917, 14569, 8890, 3754, 17854, 6166, 4669, 7950, 5372, 6525, 3957, 4852]
instance:  1
index_map [17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
target covered:  2.2699656834599615
low:  4.0
high:  8.6
instance:  2
index_map [17, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
target covered:  2.9349321013205856
low:  4.0
high:  8.6
instance:  3
index_map [17, 11, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
target covered:  3.776154678132219
low:  4.0
high:  8.6
instance:  4
index_map [17, 11, 27, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
target covered:  5.2686571150071435
low:  4.0
high:  8.6
instance:  5
index_map [17, 11, 27, 19, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
target covered:  6.5